# Start

In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

# Links:

[[citizens_st_mobile_parquet]]<br>
[[subscriptions_st_mobile]]<br>
[[citizens_dir_mobile]]<br>
[[subscriptions_st_mobile]]<br>
[[citizens_dir_mobile]]<br>
[[citizen_payments_st_mobile]]<br>
[[companies_dir_partner]]


___
# t_subscribtions_citizens_by_companies_and_cities_ch

# Links

[[citizens_st_mobile_parquet]]<br>
[[subscriptions_st_mobile]]<br>
[[entries_installation_points_dir_partner]]<br>
[[installation_point_st_partner]]<br>
[[citizen_payments_st_mobile]]<br>

In [11]:
query_text = """--sql
    CREATE TABLE db1.t_subscribtions_citizens_by_companies_and_cities_ch 
    (
        `report_date` Date,
        `city` String,
        `partner_uuid` String,
        `company_name` String,
        `partner_lk` String,
        `citizen_id_in_flat_with_subscriptions` UInt32,
        `payments_amount` UInt32,
        `activated_citizen_id` UInt32,
        `subscribed_citizen_id` UInt32,
        `flat_uuid` UInt32
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """
ch.query_run(query_text)

""


In [13]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_subscribtions_citizens_by_companies_and_cities_mv 
REFRESH EVERY 1 DAY OFFSET 5 HOUR 25 MINUTE TO db1.t_subscribtions_citizens_by_companies_and_cities_ch AS
        WITH t1 AS(SELECT
        citizens_st_mobile.report_date AS report_date,
        entries_installation_points.city AS city,
        installation_point_st.partner_uuid AS partner_uuid,
        COUNT(DISTINCT IF(citizens_st_mobile.`state` = 'activated', citizens_st_mobile.citizen_id,NULL)) as activated_citizen_id,
        COUNT(DISTINCT IF(subscriptions_st_mobile.`state` = 'activated', subscriptions_st_mobile.citizen_id,NULL)) as subscribed_citizen_id,
        COUNT(DISTINCT IF(citizens_st_mobile.`state` = 'activated', citizens_st_mobile.flat_uuid,NULL)) as flat_uuid
        FROM db1.`citizens_st_mobile_parquet_ch` AS citizens_st_mobile
        LEFT JOIN db1.`subscriptions_st_mobile_ch` AS subscriptions_st_mobile
                ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
                AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
        LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
                ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
        LEFT JOIN db1.`installation_point_st_partner_ch` AS  installation_point_st
        ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id`
        AND installation_point_st.`report_date` = citizens_st_mobile.`report_date`
        WHERE 
                citizens_st_mobile.report_date = dateTrunc('month', report_date) 
                AND citizens_st_mobile.report_date >'2023-12-31'
        GROUP BY partner_uuid,
                city,
                report_date
        ),
        --
        t2 AS (SELECT
                citizen_payments_st_mobile.report_date AS report_date,
        entries_installation_points.city AS city,
        installation_point_st.partner_uuid AS partner_uuid,
        sum(amount) AS payments_amount
        FROM db1.`citizen_payments_st_mobile_ch` AS citizen_payments_st_mobile
        LEFT JOIN db1.`citizens_st_mobile_parquet_ch` AS citizens_st_mobile 
                ON citizen_payments_st_mobile.citizen_id = citizens_st_mobile.citizen_id  
                AND citizen_payments_st_mobile.report_date = citizens_st_mobile.report_date 
        LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
                ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
        LEFT JOIN db1.`installation_point_st_partner_ch` AS installation_point_st
        ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id` 
        AND installation_point_st.`report_date` = citizen_payments_st_mobile.`report_date`
        WHERE 
                citizen_payments_st_mobile.report_date = dateTrunc('month', report_date) 
                AND citizen_payments_st_mobile.report_date >'2023-12-31'
                AND citizen_payments_st_mobile.`state` = 'success'
        GROUP BY 
                partner_uuid,
        city,
        report_date
        ),
        --
        t3 AS (SELECT
                report_date,
        city,
        partner_uuid,
                count(flat_uuid) AS citizen_id_in_flat_with_subscriptions
        FROM
                (SELECT 
                citizens_st_mobile.report_date AS report_date,
                city,
                installation_point_st.partner_uuid AS partner_uuid,
                if(citizens_st_mobile.flat_uuid !='',citizens_st_mobile.flat_uuid,null) AS flat_uuid,
                citizens_st_mobile.citizen_id AS citizen_id,
                if(subscriptions_st_mobile.state = 'activated',1,0) AS if_sub_active,
                max(if_sub_active) OVER (partition by citizens_st_mobile.flat_uuid, citizens_st_mobile.report_date ORDER BY citizens_st_mobile.report_date DESC) AS flat_with_sub_active
                FROM db1.citizens_st_mobile_parquet_ch AS citizens_st_mobile
                LEFT JOIN db1.subscriptions_st_mobile_ch AS subscriptions_st_mobile 
                        ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
                        AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
                LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
                        ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
                LEFT JOIN db1.`installation_point_st_partner_ch` AS  installation_point_st
                ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id`
                AND installation_point_st.`report_date` = citizens_st_mobile.`report_date`
                WHERE citizens_st_mobile.report_date = dateTrunc('month', report_date) 
                        AND citizens_st_mobile.report_date >'2023-12-31'
                )
        WHERE flat_with_sub_active = 1
        GROUP BY report_date,
                        city,
                        partner_uuid
        )
        --
        SELECT
                t1.report_date AS report_date,
                t1.city AS city,
                t1.partner_uuid AS partner_uuid,
                company_name,
                partner_lk,
                citizen_id_in_flat_with_subscriptions,
                payments_amount,
                activated_citizen_id,
                subscribed_citizen_id,
                flat_uuid
        FROM  t1 
        LEFT JOIN t2 ON t1.report_date = t2.report_date
                                AND t1.city = t2.city
                                AND t1.partner_uuid = t2.partner_uuid
        LEFT JOIN t3 ON t1.report_date = t3.report_date
                                AND t1.city = t3.city
                                AND t1.partner_uuid = t3.partner_uuid
        LEFT JOIN db1.companies_dir_partner_ch AS companies_dir_partner 
                                ON companies_dir_partner.partner_uuid = t1.partner_uuid
"""
ch.query_run(query_text)

""


___

# t_subscribtions_citizens_by_companies_and_cities_address_ch 

# Links

[[citizens_st_mobile_parquet]]<br>
[[subscriptions_st_mobile]]<br>
[[entries_installation_points_dir_partner]]<br>
[[installation_point_st_partner]]<br>
[[citizen_payments_st_mobile]]<br>
[[intercoms_st_partner]]<br>

In [8]:
query_text = """--sql
    CREATE TABLE db1.t_subscribtions_citizens_by_companies_and_cities_address_ch 
    (
        `report_date` Date,
        `intercom_uuid` String,
        `motherboard_id` String,
        `city` String,
        `full_address` String,
        `company_name` String,
        `partner_uuid` String,
        `partner_lk` String,
        `tin` String,
        `citizen_id_in_flat_with_subscriptions` UInt32,
        `payments_amount` UInt32,
        `activated_citizen_id` UInt32,
        `subscribed_citizen_id` UInt32,
        `flat_uuid` UInt32
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """
ch.query_run(query_text)


""


___

In [10]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_subscribtions_citizens_by_companies_and_cities_address_mv
REFRESH EVERY 1 DAY OFFSET 5 HOUR 27 MINUTE TO db1.t_subscribtions_citizens_by_companies_and_cities_address_ch AS 
		WITH citizens_st_mobile AS(
	SELECT
		report_date,
		citizen_id,
		address_uuid,
		flat_uuid,
		state
	FROM db1.`citizens_st_mobile_parquet_ch` 
	WHERE report_date = dateTrunc('month', report_date) 
		AND report_date ='2025-06-01'
		AND `state` = 'activated'
	),
	subscriptions_st_mobile AS(
	SELECT 
		report_date,
		citizen_id,
		state
	FROM db1.subscriptions_st_mobile_ch
	WHERE report_date = dateTrunc('month', report_date) 
		AND report_date ='2025-06-01'
		AND state = 'activated'
	),
	installation_point_st AS (
	SELECT 
		report_date,
		installation_point_id
	FROM db1.installation_point_st_partner_ch
	WHERE report_date = dateTrunc('month', report_date) 
		AND report_date ='2025-06-01'
	),
	intercoms_st_partner AS (
	SELECT
		report_date,
		intercom_uuid,
		partner_uuid,
		installation_point_id
	FROM db1.intercoms_st_partner_ch 
	WHERE report_date = dateTrunc('month', report_date) 
		AND report_date ='2025-06-01'
	),
	citizen_payments_st_mobile AS (
	SELECT
		report_date,
		citizen_id,
		amount
	FROM db1.citizen_payments_st_mobile_ch 
	WHERE report_date = dateTrunc('month', report_date) 
		AND report_date ='2025-06-01'
		AND state = 'success'
	),
	t4 AS(
	SELECT  
		intercoms_st_partner.report_date AS report_date,
		city,
		company_name,
		partner_lk,
		intercoms_st_partner.intercom_uuid AS intercom_uuid,
		intercoms_st_partner.partner_uuid AS partner_uuid,
		motherboard_id,
		full_address,
		tin
	FROM intercoms_st_partner
	LEFT JOIN  db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard
			ON intercoms_dir_asgard.intercom_uuid = intercoms_st_partner.intercom_uuid
	LEFT JOIN db1.companies_dir_partner_ch AS companies_dir_partner 
			ON companies_dir_partner.partner_uuid = intercoms_st_partner.partner_uuid
	LEFT JOIN installation_point_st
		ON installation_point_st.`installation_point_id` = intercoms_st_partner.`installation_point_id`
		AND installation_point_st.`report_date` = intercoms_st_partner.`report_date`
	LEFT JOIN db1.entries_installation_points_dir_partner_ch AS entries_installation_points 
			ON installation_point_st.`installation_point_id` = entries_installation_points.`installation_point_id`
	WHERE intercoms_st_partner.report_date  = dateTrunc('month', intercoms_st_partner.report_date ) 
		AND intercoms_st_partner.report_date ='2025-06-01'
	),
	--
	t1 AS (SELECT
		citizens_st_mobile.report_date AS report_date,
		intercoms_st_partner.intercom_uuid AS intercom_uuid,
		COUNT(DISTINCT if(citizens_st_mobile.citizen_id !=0, citizens_st_mobile.citizen_id ,Null)) as activated_citizen_id,	
		COUNT(DISTINCT if(subscriptions_st_mobile.citizen_id !=0, subscriptions_st_mobile.citizen_id ,Null)) as subscribed_citizen_id,	
		COUNT(DISTINCT if(citizens_st_mobile.flat_uuid !='', citizens_st_mobile.flat_uuid ,Null)) as flat_uuid
	FROM citizens_st_mobile
	LEFT JOIN  subscriptions_st_mobile
			ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
			AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
	LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
			ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
	LEFT JOIN installation_point_st
			ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id`
			AND installation_point_st.`report_date` = citizens_st_mobile.`report_date`
	LEFT JOIN intercoms_st_partner 
			ON intercoms_st_partner.installation_point_id = installation_point_st.installation_point_id 
			AND intercoms_st_partner.report_date = citizens_st_mobile.report_date 
	LEFT JOIN  db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard
			ON intercoms_dir_asgard.intercom_uuid = intercoms_st_partner.intercom_uuid
	LEFT JOIN db1.companies_dir_partner_ch AS companies_dir_partner 
			ON companies_dir_partner.partner_uuid = intercoms_st_partner.partner_uuid
	GROUP BY report_date,
			intercom_uuid
	),
	--
	t2 AS (SELECT
		citizen_payments_st_mobile.report_date AS report_date,
		intercoms_st_partner.intercom_uuid AS intercom_uuid,
		sum(amount) AS payments_amount
	FROM citizen_payments_st_mobile
	LEFT JOIN citizens_st_mobile 
			ON citizen_payments_st_mobile.citizen_id = citizens_st_mobile.citizen_id  
			AND citizen_payments_st_mobile.report_date = citizens_st_mobile.report_date 
	LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
			ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
	LEFT JOIN installation_point_st
			ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id` 
			AND installation_point_st.`report_date` = citizen_payments_st_mobile.`report_date`
	LEFT JOIN intercoms_st_partner 
			ON intercoms_st_partner.installation_point_id = installation_point_st.installation_point_id 
			AND intercoms_st_partner.report_date = citizen_payments_st_mobile.report_date
	LEFT JOIN db1.companies_dir_partner_ch AS companies_dir_partner 
			ON companies_dir_partner.partner_uuid = intercoms_st_partner.partner_uuid
	LEFT JOIN  db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard
			ON intercoms_dir_asgard.intercom_uuid = intercoms_st_partner.intercom_uuid
	GROUP BY 	
			report_date,
			intercom_uuid
	),
	--
	t3 AS (SELECT
		report_date,
		intercom_uuid,
		count(if(citizen_id !=0,citizen_id,NULL)) AS citizen_id_in_flat_with_subscriptions
	FROM
		(SELECT
			installation_point_st.report_date AS report_date,
			installation_point_st.citizen_id AS citizen_id,
			installation_point_st.installation_point_id AS installation_point_id,
			installation_point_st.flat_with_sub_active AS flat_with_sub_active,
			intercom_uuid
		FROM
			(SELECT 
				citizens_st_mobile.report_date AS report_date,
				citizens_st_mobile.citizen_id AS citizen_id,
				installation_point_st.installation_point_id AS installation_point_id,
				if(subscriptions_st_mobile.state = 'activated',1,0) AS if_sub_active,
				max(if_sub_active) OVER (partition by citizens_st_mobile.flat_uuid, citizens_st_mobile.report_date ORDER BY citizens_st_mobile.report_date DESC) AS flat_with_sub_active
			FROM citizens_st_mobile AS citizens_st_mobile
			LEFT JOIN subscriptions_st_mobile 
				ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
				AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
			LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
				ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
			LEFT JOIN db1.`installation_point_st_partner_ch` AS  installation_point_st
				ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id`
				AND installation_point_st.`report_date` = citizens_st_mobile.`report_date`) AS installation_point_st
			LEFT JOIN intercoms_st_partner 
				ON intercoms_st_partner.installation_point_id = installation_point_st.installation_point_id 
				AND intercoms_st_partner.report_date = installation_point_st.report_date
		WHERE flat_with_sub_active = 1
		)
	GROUP BY report_date,
			intercom_uuid
	)
	--
	SELECT
		t1.report_date AS report_date,
		t1.intercom_uuid AS intercom_uuid,
		t4.full_address AS full_address,
		t4.company_name AS company_name,
		t4.city AS city,
		t4.partner_uuid AS partner_uuid,
		t4.partner_lk AS partner_lk,
		t4.motherboard_id AS motherboard_id,
		t4.tin AS tin,
		payments_amount,
		activated_citizen_id,
		subscribed_citizen_id,
		flat_uuid,
		citizen_id_in_flat_with_subscriptions
	FROM  t1 
	LEFT JOIN t2 ON t1.report_date = t2.report_date
				AND t1.intercom_uuid = t2.intercom_uuid
	LEFT JOIN t3 ON t1.report_date = t3.report_date
				AND t1.intercom_uuid = t3.intercom_uuid
	LEFT JOIN t4  ON t1.report_date = t4.report_date
				AND t1.intercom_uuid = t4.intercom_uuid
	"""
ch.query_run(query_text)

""


In [11]:
query_text = """
    SELECT
        *
    FROM db1.t_subscribtions_citizens_by_companies_and_cities_address_ch
    WHERE intercom_uuid !=''
        AND full_address !=''
    ORDER BY report_date DESC
    LIMIT 10

    """
ch.query_run(query_text)

,report_date,intercom_uuid,motherboard_id,city,full_address,company_name,partner_uuid,partner_lk,tin,citizen_id_in_flat_with_subscriptions,payments_amount,activated_citizen_id,subscribed_citizen_id,flat_uuid
0,2025-06-01,3bf7ab27-7a2b-4978-8122-13f5ca1f4366,2428254646,Киров,"Россия, Кировская область, муниципальное образ...","ООО ""ЭКРАН-СЕРВИС""",c259e6bf-517a-4f87-ba0c-99c7ea8e7f44,123073,4345096371,8,0,36,3,18
1,2025-06-01,fa56bfa4-baae-4382-8caa-b96950356af4,1944103301,Тольятти,"Россия, Самарская область, городской округ Тол...",Цифрал Тольятти,e7e86c75-fc8b-4710-839d-56ad86f43211,152762,,0,0,97,0,56
2,2025-06-01,d3cc4643-6c25-4405-bb77-e8afc2e4a239,2121125492,Казань,"Россия, Республика Татарстан, городской округ ...",ЭЛЕКТРОН ПЛЮС Казань,208ab2b2-ccb9-4331-a0a1-90c539458584,120355,1657058016,16,0,66,4,30
3,2025-06-01,7418b145-1c71-4068-8adc-902b8e03e6cc,2235142681,Улан-Удэ,"Россия, Республика Бурятия, городской округ Ул...","ООО ""Мастер Домофон""",2f60241c-e1e2-4760-8a42-fd55677ac005,123688,7602067982,89,69,310,25,128
4,2025-06-01,df5e1732-94a5-4fbf-885a-e9867c777078,2208135361,Дзержинск,"Россия, Нижегородская область, городской округ...","ООО ""Сигнал""",660f767d-a49e-470d-9412-8424e09a5842,121983,5259062042,7,0,24,3,14
5,2025-06-01,add02d6c-8455-49a6-8723-288df64da95b,2121124906,Нижнекамск,"Россия, Республика Татарстан, муниципальное об...",ООО Домофон-Сервис,88334447-5e79-4866-8c01-8423c0a2f629,122326,1651048325,33,0,60,9,22
6,2025-06-01,44ea2f2b-babe-4303-a1df-c24df008d061,2148133366,Новокузнецк,"Россия, Кемеровская область, Новокузнецкий гор...",Цифрал Сервис УК,0e7236ac-b8bd-4ab0-8634-d165ad17190e,120345,,2,0,46,0,23
7,2025-06-01,bf60494c-dec1-4845-a083-cba78564073b,2101115754,Озерск,"Россия, Челябинская область, Озерский городско...",ООО Bид Плюс ПРО,963e8bcc-c6c3-46ee-958f-412c226ac672,120696,7422048677,4,69,41,1,18
8,2025-06-01,c19ecbfc-c289-4ca5-8b0f-051780a9c8b8,1921101138,Иваново,"Россия, Ивановская область, городской округ Ив...",ООО Промсвязьмонтаж ПРО,4900e775-a669-4624-b58d-916026610bba,120602,3702031990,7,0,63,3,25
9,2025-06-01,58988f1f-2ce8-43c8-b9f2-bd2cfbb77921,1944103106,Тюмень,"Россия, Тюменская область, городской округ Тюм...",ИП Менщиков С.С.,95d868e8-e80c-487f-8dbc-091dd53af79d,121832,450141847794,37,0,226,10,111


In [4]:
query_text = """
SYSTEM REFRESH VIEW db1.t_subscribtions_citizens_by_companies_and_cities_address_mv
"""

ch.query_run(query_text)

,elapsed_ns,query_id
0,1313005,942ed803-976d-4ff9-ad7e-e1a5a1b371f7


In [9]:
query_text = """
    DROP TABLE db1.t_subscribtions_citizens_by_companies_and_cities_address_mv
    """
ch.query_run(query_text)

,elapsed_ns,query_id
0,1690884,55354f8b-5c4e-4013-8e0b-9e35ef231a11
